In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from dateutil.parser import parse

In [2]:
data_version = "17-07-2020"

In [3]:
df_train = pd.read_csv("../../data/kalapa/train.csv")
df_test = pd.read_csv("../../data/kalapa/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def check_duplicate(feature1, feature2):
    check1 = list(df_train[feature1].fillna("NaN"))
    check2 = list(df_train[feature2].fillna("NaN"))
    return check1 == check2
distinct_columns = {}
for col1 in df_train.columns:
    vc = df_train[col1].value_counts()
    index = [len(vc), set(list(vc)), set(list(vc.index))]
    for col2 in distinct_columns:
        if index == distinct_columns[col2] and check_duplicate(col1, col2):
            break
    else:
        distinct_columns[col1] = index
df_train = df_train[distinct_columns]
del distinct_columns['label']
df_test = df_test[distinct_columns]

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20381 entries, 0 to 20380
Columns: 148 entries, id to Field_82
dtypes: float64(87), int64(2), object(59)
memory usage: 23.0+ MB


In [6]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    "cat_str" : ["Field_4", "Field_12", "gioiTinh", "Field_62", "Field_65", "Field_66",\
                 "info_social_sex", "Field_36", "Field_54", "Field_55", "Field_38", "brief", "Field_61", \
                 'data.basic_info.locale', "currentLocationCountry", 'homeTownCountry', "currentLocationState", \
                'homeTownState'], 
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40", \
             "Field_43", "Field_44", "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate",  ],
    "text" : ["diaChi", "Field_46", "Field_48", "Field_49", "Field_56","Field_18",\
              "maCv", "currentLocationName",'homeTownName', "currentLocationCity", "homeTownCity"],
    "drop" : [ "Field_34","Field_68", "Field_45"]
}

In [7]:
"""
a = data_type["cat_num"]+ data_type["cat_str"]+data_type["date"]+data_type["text"]+ data_type["drop"]
others = []
for feature in df_train.columns:
    if feature not in a:
        vc = df_train[feature].value_counts()
        index = [len(vc), list(vc)[:5], list(vc.index)[:5]]
        print(f"{feature}:{index}")
"""

'\na = data_type["cat_num"]+ data_type["cat_str"]+data_type["date"]+data_type["text"]+ data_type["drop"]\nothers = []\nfor feature in df_train.columns:\n    if feature not in a:\n        vc = df_train[feature].value_counts()\n        index = [len(vc), list(vc)[:5], list(vc.index)[:5]]\n        print(f"{feature}:{index}")\n'

In [8]:
"""
for column in df_train[data_type["drop"]]:
    vc = df_train[column].value_counts()
    index = [len(vc), list(vc)[:5], list(vc.index)[:5]]
    #if len(vc) <= 2 :
    print(f"{column}:{index}")
"""

'\nfor column in df_train[data_type["drop"]]:\n    vc = df_train[column].value_counts()\n    index = [len(vc), list(vc)[:5], list(vc.index)[:5]]\n    #if len(vc) <= 2 :\n    print(f"{column}:{index}")\n'

In [9]:
classes_62 = {
    "Zezo" : 0,
    "I" : 1,
    "II" : 2,
    "III": 3,
    "IV" : 4,
    "V" : 5
}

In [10]:
classes_47 = {
    "Zezo" : 0,
    "One" : 1,
    "Two" : 2,
    "Three": 3,
    "Four" : 4
}

In [11]:
def f34(date):
    try:
        res = parse(str(date)[:8]).timestamp()
        return res
    except ValueError:
        return np.nan

In [12]:
for dataset in [df_train, df_test]:
    dataset["Field_34"] = dataset["Field_34"].map(lambda x: f34(x))
    dataset["Field_47"] = dataset["Field_47"].map(lambda x: classes_47[x] if x in classes_47 else x)
    dataset["Field_62"] = dataset["Field_62"].map(lambda x: classes_62[x] if x in classes_62 else x)

In [13]:
df_test.head()

,id,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,...,partner5_A,partner5_D,partner5_G,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,53030,2019-10-14T01:37:41.959Z,2019-10-14T01:37:41.959Z,2.0,T1,8/27/2019,8/27/2019,9/24/2019,9/23/2020,8/27/2019,...,0.0,0.0,0.0,NaN,2.0,NaN,NaN,NaN,NaN,1
1,53031,2019-09-10T09:20:40.066Z,2019-09-11T07:53:44Z,2.0,T1,9/10/2019,9/10/2019,9/14/2019,9/13/2020,9/10/2019,...,1.0,0.0,0.0,notfound,1.0,NaN,NaN,NaN,NaN,1
2,53032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0,1.0,NaN,NaN,NaN,NaN,1
3,53033,2019-10-21T03:29:30.074Z,2019-10-21T04:17:26Z,2.0,T1,10/21/2019,10/21/2019,11/15/2019,11/14/2020,10/21/2019,...,0.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1
4,53034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,3,1.0,27.0,NaN,27.0,27.0,1


In [14]:
df_train.to_csv("../../data/kalapa/18-07-2020/train.csv", index = False)
df_test.to_csv("../../data/kalapa/18-07-2020/test.csv", index = False)